In [1]:
!git clone https://github.com/edu9as/dunkest.git
%cd dunkest

In [2]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from dunkest import dunkest

df = pd.read_csv("dunkest.csv", sep = ";")

In [3]:
@interact
def scatter_plot(n_days=list(range(3,df.Day.max()+1)), 
                 min_games_played=list(range(df.Day.max()+1)),
                 optimize=list(['CR', 'PDK_mean', 'PDK_sd',
                                'PDK_n', 'PDK_last', 'PDK_CR']), 
                 guards=list(range(0,10)),
                 forwards=list(range(0,10)),
                 centers=list(range(0,10))):

    pdk_cr = dunkest.build_pdk_cr(df, n_days = n_days,
                                  min_games_played=min_games_played)

    your_df = dunkest.build_best_team(pdk_cr, optimize,
                                      players_per_position=[guards,
                                                            forwards,
                                                            centers])
    del your_df["PLUS"]

    print(f"Total PDK_mean: {your_df.PDK_mean.sum():.2f}")
    print(f"Mean team PDK: {dunkest.compute_team_pdk(your_df, 'PDK_mean'):.2f}")
    print(f"Total PDK_last: {your_df.PDK_last.sum():.2f}")
    print(f"Last team PDK: {dunkest.compute_team_pdk(your_df, 'PDK_last'):.2f}")

    df_format = {k: "{:.2f}" for k in df.select_dtypes("number").columns}

    render_df = your_df.style.\
        background_gradient(subset=["PDK_sd"], cmap = "seismic").\
        bar(subset=[optimize], color = "#08b", vmin = 0).\
        format(df_format)


    return render_df


interactive(children=(Dropdown(description='n_days', options=(3, 4, 5, 6), value=3), Dropdown(description='min…